In [1]:
#!pip install pandas sqlalchemy sqlacodegen

In [2]:
import pickle
import pandas as pd

In [3]:
data = [{"name": "Justin"}]

with open("users.pkl", 'wb') as f:
    pickle.dump(data, f) # pickle.dump

In [4]:
data_from_pkl = pickle.load(open('users.pkl', 'rb')) # pickle.loads -> pickle.load
print(data_from_pkl, type(data_from_pkl))

[{'name': 'Justin'}] <class 'list'>


In [5]:
df = pd.DataFrame(data)
df.head()

,name
0,Justin


In [6]:
df.to_pickle("df_pkl")
df.dtypes

name    object
dtype: object

In [7]:
df2 = pd.read_pickle("df_pkl")
df2.head()

,name
0,Justin


In [8]:
df2.dtypes

name    object
dtype: object

In [9]:
df.to_csv("my_df.csv", index=False)

SQLAlchemy
SQLAlchmey & Django have a similar object-relational mappings (aka ORM) are ways to connect Python to a SQL database.

In [10]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str = 'Unknown'
    genre:str = 'Action'
    year:int = None
    
# class Movie:
#     name = 'Unknown'
#     genre = 'Action'
    
#     def __init__(self, name='', *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.name = name

In [11]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
movie_obj.name

'Interstellar'

In [12]:
data = [{
    "name": "Interstellar",
    "genre": "Sci-Fi"
},
{
    "name": "The Martian",
    "genre": "Sci-Fi"
},
{
    "name": "Arrival",
    "genre": "Sci-Fi"
}
]

In [37]:
df = pd.DataFrame(data)
df.head()

,name,genre
0,Interstellar,Sci-Fi
1,The Martian,Sci-Fi
2,Arrival,Sci-Fi


In [38]:
import sqlalchemy
from sqlalchemy import Column, Integer, String
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [39]:
engine = create_engine("sqlite:///app.db") # mysql, postgres

In [40]:
Session = sessionmaker(bind=engine)
my_sess = Session()

# why not this way? :

#my_sess = sessionmaker(bind=engine)

# Because it throw an error......

In [41]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [42]:
class Movie(Base): # table
    __tablename__ = "movies"
    
    id = Column(Integer, primary_key=True) # auto created for us
    name = Column(String) # 'Unknown' # col
    genre = Column(String) # 'Action' # col
    description = Column(String) # 'Action' # col
    year = Column(Integer, nullable=True) # None # col
    
    
    def __repr__(self):
        return f"<Movie name={self.name}>"
        
# saved instance = row

In [43]:
# Add table(s) to database
Base.metadata.create_all(engine)

In [44]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
print(movie_obj.name)

Interstellar


In [45]:
movie_obj.id

In [46]:
my_sess.add(movie_obj) # prepare to save
my_sess.commit() # save

In [47]:
print(movie_obj.id, movie_obj.description)

4 None


In [48]:
movie_obj2 = Movie(name='The Martian', genre='Sci-Fi')
print(movie_obj2.name, movie_obj2.id)

The Martian None


In [49]:
my_sess.add(movie_obj2)

In [50]:
movie_obj3 = Movie(name='Inception', genre='Sci-Fi')
print(movie_obj3.name, movie_obj3.id)

Inception None


In [51]:
my_sess.add(movie_obj3)

In [52]:
my_sess.commit()

In [53]:
print(movie_obj2.id, movie_obj3.id)

5 6


# CRUD in SQLAlchemy
Create
Retreive
Update
Delete

In [54]:
Session = sessionmaker(bind=engine)
session = Session()

# Create

In [55]:

movie = Movie(name='Guaridans of the Galaxy')
session.add(movie)
session.commit()

# Retreive

## Get 1 item

In [56]:
movie_a = session.query(Movie).get(5)
print(movie_a.id, movie_a.name, movie_a.description)

5 The Martian None


## List

In [57]:
qs = session.query(Movie).all()
print(qs)

[<Movie name=Interstellar>, <Movie name=The Martian>, <Movie name=Inception>, <Movie name=Interstellar>, <Movie name=The Martian>, <Movie name=Inception>, <Movie name=Guaridans of the Galaxy>]


## List & Filter by Column Value

In [58]:
qs = session.query(Movie).filter_by(name='Guardians of the Galaxy').all()
qs

[]

## List & Filter by Column Value Containing Something

In [59]:

qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[<Movie name=Guaridans of the Galaxy>]

## List & Filter by Column Value Containing Something

In [60]:
my_query = input("What are you looking for?\n") or "Unknown"
qs = session.query(Movie).filter(Movie.name.contains(my_query)).all()
print(qs)

What are you looking for?
gua
[<Movie name=Guaridans of the Galaxy>]


# Update

In [61]:
movie_a = session.query(Movie).get(5)
movie_a.description = "A dream within a dream"
print(movie_a.id, movie_a.name, movie_a.description)
session.commit()

5 The Martian A dream within a dream


In [62]:
movie_a = session.query(Movie).get(5)
print(movie_a.description)

A dream within a dream


In [63]:
qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
for movie_obj in qs:
    movie_obj.name = "Guardians of the Galaxy"
session.commit()

In [64]:
qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[]

# Delete

In [65]:
movie_a = session.query(Movie).get(1)
session.delete(movie_a)
session.commit()

In [66]:
session.flush()

#SQLAlchemy & Pandas

In [68]:
!sqlacodegen sqlite:///app.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)
